See 1. A Numerical Example in Valuing American Options by Simulation: A Simple Least-Squares Approach Francis A. Longstaff UCLA Eduardo S. Schwartz UCLA.

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

paths = np.array([[1.00, 1.09, 1.08, 1.34],
                 [1.00, 1.16, 1.26, 1.54],
                 [1.00, 1.22, 1.07, 1.03],
                 [1.00, 0.93, 0.97, 0.92],
                 [1.00, 1.11, 1.56, 1.52],
                 [1.00, 0.76, 0.77, 0.90],
                 [1.00, 0.92, 0.84, 1.01],
                 [1.00, 0.88, 1.22, 1.34]])

def least_square_Monte_Carlo_test(paths,r,dt,T,payoff,K):
    N = round(T/dt)
    value = payoff(paths[:,-1])
    print(value)

    for i in range(N-1): # stepping backwards in time 
       value = value*np.exp(-r*dt) # discounting 
       ITMmask = paths[:,-i-2] < K
       print(f"mask {ITMmask}")
       ITMpaths = paths[ITMmask]
       ITMvalue = value[ITMmask]
       model = LinearRegression()
       X = np.column_stack((ITMpaths[:,-i-2],ITMpaths[:,-i-2]**2))
       model.fit(X,ITMvalue)
       print(f"model.coef_ {model.intercept_} {model.coef_}")
       interpolated = model.predict(X)
       exercise = payoff(ITMpaths[:,-i-2])
       print(f"interpolated {interpolated}")
       print(f"exercise {exercise}")
    #  value[mask] = np.maximum(interpolated,exercise) see note in paper
       value[ITMmask] = np.where(interpolated>exercise, value[ITMmask], exercise)

    return np.exp(-r*dt)*np.mean(value), np.exp(-r*dt)*np.std(value)/np.sqrt(len(value))
    
def payoff_put(S,K): return np.maximum(K-S,0)

T = 3
dt =1 
r = 0.06
K = 1.10

val,std = least_square_Monte_Carlo_test(paths,r,dt,T,lambda S: payoff_put(S,K),K)
print(val,std)

[0.   0.   0.07 0.18 0.   0.2  0.09 0.  ]
mask [ True False  True  True False  True  True False]
model.coef_ -1.0699876552910912 [ 2.98341063 -1.81357618]
interpolated [0.03674056 0.04589834 0.11752682 0.15196921 0.15641792]
exercise [0.02 0.03 0.13 0.33 0.26]
mask [ True False False  True False  True  True  True]
model.coef_ 2.037512342379653 [-3.3354434   1.35645659]
interpolated [0.01348511 0.10874928 0.28606468 0.11700927 0.15276213]
exercise [0.01 0.17 0.34 0.18 0.22]
0.11443433004505696 0.03922691623092561
